# Construct a Simple RAG based on Vector Database out of text using a LLM

```bash
! pip install llama-index
! pip install llama-cpp-python
! pip install transformers
! pip install torch


```

In [13]:
####### load chunked data
import re
import json
import os

with open("./chunks.json", "r") as f:
    chunks = json.loads(f.read())
print(len(chunks)) #document chunks are nodes in LlamaIndex

97


In [14]:
####### Build Nodes from chunked data
# A Node represents a “chunk” of a source Document, whether that is a text chunk, an image, or other. 
# Similar to Documents, they contain metadata and relationship information with other nodes.
from llama_index.schema import TextNode
nodes = []
for chunk in chunks:
    metadata = chunk['kwargs']['metadata']
    content = chunk['kwargs']['page_content']
    #content = re.sub('[^A-Z.a-z]+', ' ', content)
    nodes.append(TextNode(text=content, id_=metadata['uuid'], metadata=metadata))
print(len(nodes))
print(nodes[0])
print(nodes[0].metadata)

97
Node ID: 0af897c6-3391-496e-a93c-a5a18b350f50
Text: Albert Einstein ( EYEN-styne; German: [ˈalbɛɐt ˈʔaɪnʃtaɪn] ; 14
March 1879 – 18 April 1955) was a German-born theoretical physicist
who is widely held to be one of the greatest and most influential
scientists of all time. Best known for developing the theory of
relativity, Einstein also made important contributions to quantum
mechanics, and was ...
{'doc_source': 'wikipedia', 'wikipedia_page': 'Albert Einstein', 'doc_lang': 'en', 'doc_size': 83894, 'timestamp': 1702572674.745187, 'uuid': '0af897c6-3391-496e-a93c-a5a18b350f50'}


In [15]:
######### Load LLM
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
import os

#model = "llama-2-13b-chat.Q5_K_M.gguf"
model = "mistral-7b-openorca.Q5_K_M.gguf" #https://huggingface.co/TheBloke/Mistral-7B-OpenOrca-GGUF
llama_cpp_path = "/Users/done/Documents/dneumnn/llama.cpp/models"
model_path = os.path.join(llama_cpp_path, model)

llm = LlamaCPP(
    model_url=None,
    model_path=model_path,
    temperature=0.1,
    max_new_tokens=2000,
    context_window=4000,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /Users/done/Documents/dneumnn/llama.cpp/models/mistral-7b-openorca.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  4096, 32002,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q5_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q5_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q5_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q5_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q5_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    7:            blk.0.ffn_down.weig

In [16]:
# use Huggingface embeddings
# https://huggingface.co/BAAI/bge-small-en-v1.5
from llama_index.embeddings import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

# create a service context
from llama_index import ServiceContext
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
)
from llama_index import VectorStoreIndex
index = VectorStoreIndex(nodes, service_context=service_context, show_progress=True)

# Persist the index to disk
storage_directory = "./"
index.storage_context.persist(persist_dir=storage_directory)

ImportError: 
AutoModel requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.


In [34]:
######### test model
response = llm.complete("Please give me a short summary of the life of Maja Einstein, the sister of Albert Einstein.", temperature=0)
print(response.text)

Llama.generate: prefix-match hit




Maja Einstein, born Mileva Marić, was the first wife and closest scientific collaborator of Albert Einstein. She was born on January 19, 1876, in Novi Sad, Austria-Hungary (now Serbia). Maja studied at the University of Zurich, where she met Albert Einstein in 1896. They got married in 1903 and had two children together.

Maja contributed to Albert's work by helping him with mathematical calculations and problem-solving. She was particularly interested in physics and mathematics, which led her to collaborate on some of his early papers. However, her contributions were often not recognized or acknowledged due to gender bias during that time period.

In 1914, Maja and Albert separated, and their marriage ended in divorce in 1919. After the divorce, she continued to live in Zurich with their two sons. She passed away on January 20, 1976, at the age of 100.

Maja Einstein's life was marked by her significant contributions to Albert Einstein's work and her resilience in the face of advers


llama_print_timings:        load time =     482.90 ms
llama_print_timings:      sample time =      92.92 ms /   262 runs   (    0.35 ms per token,  2819.54 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    6762.93 ms /   262 runs   (   25.81 ms per token,    38.74 tokens per second)
llama_print_timings:       total time =    7921.58 ms


In [36]:
from IPython.display import Markdown, display
from llama_index.prompts import PromptTemplate

query_engine = index.as_query_engine(service_context=service_context,
                                     similarity_top_k=3)


In [38]:
response = query_engine.query("Please give me a short summary of the life of Maja Einstein, the sister of Albert Einstein.")
print(response.get_formatted_sources())

Llama.generate: prefix-match hit


> Source (Doc id: 96d9e34e-a973-4db6-894d-5aeb9a192d35): the age of twenty, Einstein's son Eduard was diagnosed with schizophrenia. He spent the remainder...

> Source (Doc id: 0af897c6-3391-496e-a93c-a5a18b350f50): Albert Einstein ( EYEN-styne; German: [ˈalbɛɐt ˈʔaɪnʃtaɪn] ; 14 March 1879 – 18 April 1955) was a...

> Source (Doc id: 57187cee-a334-4c5f-bfab-4551cf726c95): == Life and career ==


=== Childhood, youth and education ===



llama_print_timings:        load time =     482.90 ms
llama_print_timings:      sample time =      23.91 ms /    78 runs   (    0.31 ms per token,  3262.23 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    2214.68 ms /    78 runs   (   28.39 ms per token,    35.22 tokens per second)
llama_print_timings:       total time =    2482.02 ms


In [ ]:
display(Markdown(f"<b>{response}</b>"))

In [ ]:
##########

In [ ]:
from llama_index import set_global_tokenizer
from transformers import AutoTokenizer

set_global_tokenizer(
    AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf").encode
    #AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf").encode
)
